# Kurztests SWK5UE WS23/24


## Kurztest 1 - Nachtermin

### Ski-Rennen - Ergebnisauswertung
Gegeben sind folgende Typen, die das Ergebnis eines Ski-Rennens abbilden.

In [2]:
public record Result(
    string Racer,       // Name Teilnehmer:in
    long Time,          // benötigte Zeit in Millisekunden
    bool Finished       // Rennen erfolgreich beendet?
);

public record RankingInfo(string Racer, long Time);

public class SkiRace
{
    protected List<Result> results = new(); // bereits befüllt

    public void AddResult(string racer, long time, bool finished)
    {
        results.Add(new Result(racer, time, finished));
    }

	public List<RankingInfo> GetRanking()
	{
		var res = 	from result in this.results
					where result.Finished == true
					orderby result.Time
					select new RankingInfo
					(
						result.Racer,
						result.Time
					);

		return res.ToList<RankingInfo>();
	}
}


### Beispiel
a) Komplettieren Sie die gegebene Methode GetRanking(Klasse SkiRace), welche die (nach Zeit aufsteigend sortierte) Rangliste für das Rennen ermittelt. Es sollen nur Teilnehmer:innen berücksichtigt werden, die das Rennen erfolgreich beendet haben (Finished).

Verwenden Sie dazu so weit wie möglich LINQ (Query- oder/und Method-Syntax).

Hinweis: Sie können davon ausgehen, dass keine Zeit (Time) mehrfach vorkommt.

```
public record RankingInfo(string Racer, long Time);

public List<RankingInfo> GetRanking()
{
```

In [3]:
SkiRace race = new();

race.AddResult("Franz Klammer", 10, true);
race.AddResult("Sepp Forcher", 13, true);
race.AddResult("Hansi Hinterseer", 14, true);
race.AddResult("Patrik Ortlieb", 12, true);
race.AddResult("Hermann Maier", 11, false);

var results = race.GetRanking();
foreach (var result in results)
{
	Console.WriteLine(result);
}


RankingInfo { Racer = Franz Klammer, Time = 10 }
RankingInfo { Racer = Patrik Ortlieb, Time = 12 }
RankingInfo { Racer = Sepp Forcher, Time = 13 }
RankingInfo { Racer = Hansi Hinterseer, Time = 14 }


```}```

### Beispiel
b) Implementieren Sie eine Erweiterungsmethode (Extension Method) **GetRacersWithPoints** für den Typ SkiRace, welche auf Basis der Daten von GetRanking die Namen der Läufer:innen liefert, welche Weltcup-Punkte erreicht haben (> 0 Punkte).

Als Argument wird eine Delegate-Instanz vom Typ Func<int, int> übergeben, welche für eine Renn-Position (Platzierung) die entsprechende Weltcup-Punkte liefert.

Die Verwendung soll folgendermaßen möglich sein:
```
Func<int, int> mapPositionToPoints = ...; // gegeben

var skiRace = new SkiRace();

IEnumerable<string> racers = skiRace.getRacersWithPoints(mapPositionToPoints);

```

In [4]:
public int mapPositionToPoints(int ranking)
{
    switch (ranking)
    {
        case 1:	return 100;
        case 2: return 80;
    	case 3: return 60;
        // case 4: return 50;
        // case 5: return 45;
        // case 6: return 40;
        // case 7: return 36;
        // case 8: return 32;
        // case 9: return 29;
        // case 10: return 26;

        default: return 0; // Return 0 points for ranks beyond 10th place
    }
}

static IEnumerable<string> GetRacersWithPoints (this SkiRace race, Func<int, int> arg)
{
	List<string> racersWithPoints = new();
	var results = race.GetRanking();
	int idx = 1;
	foreach (var racer in results)
	{
		if (arg(idx) > 0) {
			racersWithPoints.Add(racer.Racer);
		}
		idx++;
	}
	return racersWithPoints;
}

IEnumerable<string> racers = race.GetRacersWithPoints(mapPositionToPoints);

foreach (var racer in racers)
{
	Console.WriteLine(racer);
}



Franz Klammer
Patrik Ortlieb
Sepp Forcher


### Beispiel
c) Implementieren Sie die **asynchrone** Methode ``PrintCertificatesAsync`` (für die Klasse SkiRace), welche für jede(n) erfolgreiche(n) Läufer:in (auf Basis der Daten von GetRanking) unter Zuhilfenahme der Klasse ``Printer``(gegeben) jeweils eine Urkunde ausdruckt. Aufrufer von ``PrintCertificatesAsync`` sollen die Anzahl der gedruckten Urkunden mitgeteilt bekommen.

```
public class Printer : IDisposable {
    public Task PrintAsync(string racer) {...}
}
```

In [5]:
public class Printer : IDisposable {
	public void Dispose() { }
    public async Task PrintAsync(string racer) 
	{
        Console.WriteLine($"Printing racer: {racer}");
        await Task.Delay(1000); // Simulate printing delay
        Console.WriteLine($"Printed racer: {racer}\n");
	}
}

static async Task<int> PrintCertificatesAsync(this SkiRace race)
{
	var racers = race.GetRanking();
	int cnt = 0;
	using (var printer = new Printer())
	{
		foreach (var racer in racers) {
			await printer.PrintAsync(racer.Racer);
			cnt++;
		}
	}
	return cnt;
}

var count = await race.PrintCertificatesAsync();
Console.WriteLine($"{count} certificates printed.")

Printing racer: Franz Klammer
Printed racer: Franz Klammer

Printing racer: Patrik Ortlieb
Printed racer: Patrik Ortlieb

Printing racer: Sepp Forcher
Printed racer: Sepp Forcher

Printing racer: Hansi Hinterseer
Printed racer: Hansi Hinterseer

4 certificates printed.


## Kurztest 2

### Beispiel
Hier fehlt die Angabe (nicht im Screenshot). Glaube die Angabe war zirka so.
In der Tabelle **tbl_person(id: int, name: varchar(255))** sind Personen gespeichert. Lesen Sie mittels der gegebenen Methode **AdoTemplate.Query** die Personen aus **tbl_person** aus und speichern Sie diese in **allPersons**.

```
record Person(int Id, string Name);

public delegate T RowMapper<T>(IDataRecord row);

public class AdoTemplate
{
    public IEnumerable<T> Query<T>(string sql, RowMapper<T> rowMapper) { ... }
}

IEnumerable<Person> allPersons = ...
```

In [6]:
using System.Data;
record Person(int Id, string Name);
public delegate T RowMapper<T>(IDataRecord row);

private Person MapRowToPerson(IDataRecord row)
=> new Person(	(int)row["id"],
				(string)row["name"]);

public class AdoTemplate
{
    public IEnumerable<T> Query<T>(string sql, RowMapper<T> rowMapper) 
	{
		return Enumerable.Empty<T>();
	}
}			

AdoTemplate template = new();
IEnumerable<Person> allPersons = template.Query<Person>("select * from tbl_person", MapRowToPerson);

// (Query-Anfrage und Rowmapper sollten gemacht werden).

### Beispiel
Gegeben ist eine Liste von Dateien:

```
public class File
{
    public string Path { get; set; }
    public long SizeInBytes { get; set; }
}

IEnumerable<File> files = GetFiles();

```

Geben Sie die Namen aller Dateien, die größer als 1 MB sind, alphabetisch sortiert auf die Konsole aus. Verwenden Sie dazu soweit wie möglich LINQ.

In [7]:
public class File
{
    public string Path { get; set; }
    public long SizeInBytes { get; set; }
}

public static IEnumerable<File> GetFiles()
{
	return new List<File>
	{
		new File { Path = "file1.txt", SizeInBytes = 500000 }, // 0.5 MB
		new File { Path = "file2.txt", SizeInBytes = 1500000 }, // 1.5 MB
		new File { Path = "file3.txt", SizeInBytes = 2000000 } // 2 MB
	};
}

IEnumerable<File> files = GetFiles();

public static void WriteFilesLargerThan1MB(IEnumerable<File> files)
{
	var largerThan1MBFiles = from file in files
								where file.SizeInBytes > 1024 * 1024 // 1 MB = 1024 * 1024 bytes
								orderby file.Path
								select file.Path;

	Console.WriteLine("Files larger than 1MB:");
	foreach (var filePath in largerThan1MBFiles)
	{
		Console.WriteLine(filePath);
	}
}

WriteFilesLargerThan1MB(files);



Files larger than 1MB:
file2.txt
file3.txt


### Beispiel
Eine Klasse **Person** ist folgendermaßen definiert:

```
delegate void BirthdayHandler(int age);

public class Person 
{
    // 
    // TODO (1a)
    //
    public DateTime Birthday { get; init; }

    private void CheckBirthday()
    {
        bool hasBirthday = DateTime.Now.DayOfYear == BirthDay.DayOfYear;
        int age = DateTime.Now.Year - BirthDay.Year;
        //
        // TODO (1b)
        //
    }

}

```

1. Fügen Sie zu **Person** ein Event **OnBirthday** hinzu (1a) und komplettieren Sie die Methode **CheckBirthday** (1b), welche einmal pro Tag aufgerufen wird und das Event feuern soll, wenn die Person an diesem Tag Geburtstag hat.
2. Registrieren Sie sich bei einer Person und gratulieren Sie dieser mittels einer Konsolenausgabe (mit Altersangabe), sobald diese Geburtstag hat.

In [ ]:
public delegate void BirthdayHandler(int age);

public class Person 
{
    // TODO (1a)
	public event BirthdayHandler onBirthday;

    public DateTime Birthday { get; init; }

    public void CheckBirthday()
    {
        bool hasBirthday = DateTime.Now.DayOfYear == Birthday.DayOfYear;
        int age = DateTime.Now.Year - Birthday.Year;
		//Console.WriteLine($"{DateTime.Now.DayOfYear}, {Birthday.DayOfYear}");
        // TODO (1b)
		if (hasBirthday && onBirthday != null)
		{
			onBirthday(age);	
		}
    }
}

Person person = new Person { Birthday = new DateTime(1990, 3, 19) }; // Example birthday

person.onBirthday += (int age) => Console.WriteLine($"Happy Birthday! You are now {age} years old.");

// person.onBirthday += CongratulateOnBirthday;
// static void CongratulateOnBirthday(int age)
// {
// 	Console.WriteLine($"Happy Birthday! You are now {age} years old.");
// }

while (true)
{
	person.CheckBirthday();
	System.Threading.Thread.Sleep(TimeSpan.FromSeconds(1));
}




Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 years old.
Happy Birthday! You are now 34 yea

### Beispiel
Gehen Sie von folgendem Controller aus:
```
[Route("warehouse/[controller]")]
public class ArticlesController : ControllerBase
{
    [HttpGet("{id}")]
    public ArticleDTO GetArticle(int id) { ... }
}
```
Mit welcher URL kann die Methode GetProduct angesprochen werden?

- [x] https://localhost:5000/warehouse/articles/123
- [ ] https://localhost:5000/api/articles/123
- [ ] https://localhost:5000/warehouse/123
- [ ] https://localhost:5000/warehouse

### Beispiel
Hierfür habe ich leider die Angabe nicht fotografiert - es war in etwa: Gegeben ist folgende Klasse **Day**
```
{
    public DayViewModel SelectedDay { get; set; }
    public IEnumerable<DayViewModel> Days { get; set; }
}
```

Welchen Datentyp hat der **DataContext** an der markierten Stelle?
```
<Window>
    <ListBox ItemSource="{Binding Days}">
        <ListBox.ItemTemplate>
            <DataTemplate>
                <!-- DataContext? -->
            </DataTemplate>
        </ListBox.ItemTemplate>
    </ListBox>
</Window>
```
- [ ] IEnumerable<DayViewModel>
- [ ] Days
- [ ] object
- [ ] ListBoxItem
- [ ] CalendarViewModel
- [ ] SelectedDays
- [x] DayViewModel

### Beispiel

ApiController: Service wird folgender Request geschickt:

POST api/articles
Content-Type: application/json
{
    "ArticleCode": "A7615",
    "Description": "Book: C# for Dummies"
}

- Implementieren Sie eine dazu passende Controller Methode inklusive der erforderlichen Attribute (nur die Methode, keine Klasse erforderlich).
- Sollte es bereits einen Artikel mit dem selben ArticleCode geben, soll der Client mittels Statuscode (409 Conflict) benachrichtigt werden.
- Liefern Sie im ERfolgsfall einen StatusCode 201 (oder 200) mit leerem Body zurück - der Location-Header muss nicht gesetzt sein.

Sie können folgenden Code und Geschäftslogikmethoden, sowie eine Instanz von IArticleLogic voraussetzen:

```
public class Article
{
    public string ArticleCode { get; set; }
    public string Description { get; set; }
}

public interface IArticleLogic
{
    Task<bool> ArticleExistsAsync(string articleCode);
    Task CreateArticleAsync(Article article);
}

```

In [1]:
#r "nuget: Microsoft.AspNetCore.Mvc"
public class Article
{
    public string ArticleCode { get; set; }
    public string Description { get; set; }
}

public interface IArticleLogic
{
    Task<bool> ArticleExistsAsync(string articleCode);
    Task CreateArticleAsync(Article article);
}

Installed Packages Microsoft.AspNetCore.Mvc, 2.2.0

In [2]:
using Microsoft.AspNetCore.Mvc;

public class ArticleController : ControllerBase
{
	private readonly IArticleLogic aLogic;

	[HttpPost]
	public async Task<ActionResult> CreateArticle([FromBody] Article article)
	{
		if (await aLogic.ArticleExistsAsync(article.ArticleCode))
		{
			return Conflict();
		}

		await aLogic.CreateArticleAsync(article);
		return Ok();
	}
}